<a href="https://colab.research.google.com/github/Beebruna/API-TRAINING/blob/main/doe4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalação python 3.10

In [10]:
!pip install fake-useragent

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-py3-none-any.whl size=13502 sha256=920bc43479d3e4dbb8b188126c45a6605b08088073ce832eda9e5fbc9b33c4d6
  Stored in directory: /root/.cache/pip/wheels/ed/f7/62/50ab6c9a0b5567267ab76a9daa9d06315704209b2c5d032031
Successfully built fake-useragent


In [32]:
!pip install cfscrape

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!sudo apt install python3.10

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  libpython3.10-minimal libpython3.10-stdlib python3.10-minimal
Suggested packages:
  python3.10-venv binfmt-support
The following NEW packages will be installed:
  libpython3.10-minimal libpython3.10-stdlib python3.10 python3.10-minimal
0 upgraded, 4 newly installed, 0 to remove and 20 not upgraded.
Need to get 5,093 kB of archives.
After this operation, 19.5 MB of additional disk space will be used.
Get:1 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic/main amd64 libpython3.10-minimal amd64 3.10.6-1+bionic1 [823 kB]
Get:2 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic/main amd64 python3.10-minimal amd64 3.10.6-1+bionic1 [1,968 kB]
Get:3 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bion

In [ ]:
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.10 1

In [ ]:
!sudo update-alternatives --config python3

There are 3 choices for the alternative python3 (providing /usr/bin/python3).

  Selection    Path                 Priority   Status
------------------------------------------------------------
* 0            /usr/bin/python3.7    2         auto mode
  1            /usr/bin/python3.10   1         manual mode
  2            /usr/bin/python3.6    1         manual mode
  3            /usr/bin/python3.7    2         manual mode

Press <enter> to keep the current choice[*], or type selection number: 1
update-alternatives: using /usr/bin/python3.10 to provide /usr/bin/python3 (python3) in manual mode


In [ ]:
!python3 --version

Python 3.10.6


# DOE

In [37]:
import time
import cfscrape
import requests
import pandas as pd
from tqdm import tqdm
from random import randint
from bs4 import BeautifulSoup
from fake_useragent import UserAgent

In [26]:
def get_userAgent():
    #'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'
    ua = UserAgent()
    return ua.random

In [48]:
def get_html(url: str) -> BeautifulSoup:
    '''
    headers = {'user-agent': f'{get_userAgent()}',
               'referer':'https://www.ioepa.com.br/Frame',
               'accept-encoding': 'gzip, deflate, br',
               'accept-language': 'pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7',
               'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9'
               }
    html = requests.get(url, headers=headers)
    '''
    cf_scrape = cfscrape.create_scraper()
    response = cf_scrape.get(url)
    print('Response 1:', response)
    dom = BeautifulSoup(response.text, 'html.parser')
    return dom

In [43]:
cf_scrape = cfscrape.create_scraper()
response = cf_scrape.get('https://www.youtube.com/')

In [47]:
response.url

'https://www.youtube.com/'

In [3]:
def get_link(html: BeautifulSoup) -> list:
    return [tag_a['href'] for tag_a in html.find_all('a')]

In [4]:
def verify_doe(html: BeautifulSoup) -> bool:
    status = html.find('div', class_="alert alert-danger")
    return bool(status)

In [49]:
def download_file_from_google_drive(link: str, destination: str):
    URL = "https://docs.google.com/uc?export=download"

    id = id = link.split('/')[5]
    '''
    headers = {'user-agent': f'{get_userAgent()}',
               'referer':'https://www.ioepa.com.br/Frame',
               'accept-encoding': 'gzip, deflate, br',
               'accept-language': 'pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7',
               'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9'
               }
    session = requests.Session()
    response = session.get(URL, params = { 'id' : id }, stream = True, headers=headers)
    '''
    cf_scrape = cfscrape.create_scraper()
    response = cf_scrape.get(URL + '&id=' + id)
    print('Response 2:', response)
    '''
    token = get_confirm_token(response)
    
    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True, headers=headers)
        print('Response 3:', response)
    '''
    save_file(response, destination)

In [6]:
def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value
    return None

In [7]:
def save_file(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
        print('download: ', response.url)

In [41]:
dates = pd.date_range("20210101", pd.Timestamp.today())

In [52]:
type(dates[0].strftime('%d/%m/%Y'))

str

In [54]:
no_post = list()
count = 0
bot = 0
for data in tqdm(dates):
    html = get_html('https://www.ioepa.com.br/Frame/?dts=' + data.strftime('%d/%m/%Y'))

    if verify_doe(html): #NONE = existe o documento
        no_post.append(data)
        continue
    
    count += 1
    bot += 1
    link = get_link(html)

    if bot == 7:
        bot = 0
        time.sleep(20)

    for url in link:
        download_file_from_google_drive(url, '/content/drive/MyDrive/Contratação - Desafio/does/' + data.strftime('%d-%m-%Y') + '.pdf')
    
    print(data.date())
    time.sleep(randint(3, 20))

print(f'Documentos baixados: {count}')
print(f'Não existe documentos: {len(no_post)}')

  0%|          | 1/610 [00:00<05:37,  1.81it/s]

Response 1: <Response [200]>


  0%|          | 2/610 [00:01<05:36,  1.81it/s]

Response 1: <Response [200]>


  0%|          | 3/610 [00:01<05:36,  1.81it/s]

...

Response 1: <Response [200]>
Response 1: <Response [200]>
Response 2: <Response [403]>
download:  https://doc-14-ak-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/g2dpe6ekk91hjetn1f3rok46e2d4mucl/1662125475000/15184588300033342989/*/1kJfJjBtqsVg2kXmpw1Hytj08dBjKIBvm?e=download&uuid=f0d8379c-2984-4dae-9740-2e5901670627
2021-02-08


  6%|▋         | 39/610 [08:04<2:27:39, 15.52s/it]

Response 1: <Response [200]>
Response 2: <Response [403]>
download:  https://doc-04-ak-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ns2p93i63nq92g1ac8mrqnosei98jrge/1662125550000/15184588300033342989/*/1AQCT-20Rl3b04Z4e1OWzJErbKGDhVjnn?e=download&uuid=5b115f7e-2314-4792-b9df-7705e2854c09


  6%|▋         | 39/610 [08:37<2:06:10, 13.26s/it]


KeyboardInterrupt: ignored